# Homework 3 - What is the best anime in the world?

- Martina Milazzo
- Dimitri Saliola
- Roberta Giorgi




## 1. Data collection

In [ ]:
import requests as r
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import os
from datetime import datetime
import time
import csv
import numpy as np
import pandas as pd

### 1.1. Get the list of animes

In [ ]:
#383 pages * 50 animes (19130 total)

with open("C:/Users/marti/Desktop/HW3/list_anime.txt", "w", encoding='utf-8') as file:
    
    for page in tqdm(range(0, 383)):
    
        url = 'https://myanimelist.net/topanime.php?limit=' + str(page*50) #single page URL
        response = r.get(url)
        soup = BeautifulSoup(response.text, 'html.parser') #get html

        #extract all the links
        for tag in soup.find_all("tr", class_="ranking-list"):
            a_list = tag.find_all('a', class_="hoverinfo_trigger fl-l ml12 mr8" ,href=True)
            for a in a_list:
                link = a['href']
                file.write(str(link) + '\n')
                
file.close()

100%|██████████| 383/383 [05:05<00:00,  1.25it/s]


### 1.2. Crawl animes

In [ ]:
#DO NOT EXECUTE THIS CELL#

file = open("C:/Users/marti/Desktop/HW3/list_anime.txt", "r", encoding='utf-8')

pages_dir="C:/Users/marti/Desktop/HW3/Pages"

if not os.path.exists(pages_dir):
    os.makedirs(pages_dir)

counter_anime = 855
page = 18

for line in file:

    counter_anime += 1
    if (counter_anime%50 == 1):
        page +=1

    request = r.get(line, headers={'Cache-Control': 'no-cache'})
    #code=request.status_code
    if str(request.status_code)[0] == '4': #CODE
        while(True): #while(code[0]!='2'):
            time.sleep(120)
            request = r.get(line, headers={'Cache-Control': 'no-cache'})
            print(request.status_code)
            if str(request.status_code)[0] == '2': #questa da eliminare
                break       #questa pure
            

    soup = BeautifulSoup(request.text, 'html.parser')

    folder_dir=pages_dir+"/Page"+str(page)
    if not os.path.exists(folder_dir):
        os.makedirs(folder_dir)

    txt_dir=folder_dir + "/article_" + str(counter_anime)+ ".html"
    
    with open(txt_dir, "w", encoding='utf-8') as file:
        file.write(str(soup))

In [13]:
# Count anime links in file

file = open("C:\\Users\\marti\\Desktop\\HW3\\list_anime.txt", "r", encoding='utf-8')
line_count = 0
for line in file:
    if line != "\n":
        line_count += 1
file.close()

print(line_count)

19131


### 1.3 Parse downloaded pages

In [ ]:
#DO NOT EXECUTE THIS CELL#

title = []
types = []
numEpisode = []
release = []
end = []
numMembers = []
score = []
users = []
rank = []
popularity = []
description = []
related = []
characters = []
voices = []
staff = []

pages = os.listdir("C:/Users/marti/Desktop/HW3/Pages")[1:]

for p in pages:
    n_html = os.listdir(pages+"/"+str(p))
    for i in range(1, len(n_html)+1):
        file = open(pages+"/"+str(p)+"/article_"+str(i), 'r', encoding="utf8")
        anime = BeautifulSoup(file, 'lxml')
        
        #TITLE
        try:
            t=anime.strong.contents[0]
            title.append(t)
        except:
            title.append('NA')
            

        #TYPE
        try:
            ty=anime.find(text = 'Type:').find_next('a').contents[0]
            types.append(ty)
        except:
            types.append('NA')


        #NUM_EPISODE
        try:
            n=int(anime.find(text = 'Episodes:').next_element.strip())
            numEpisode.append(n)
        except:
            numEpisode.append(n)


        #RELEASE_DATE
        try:
            date = anime.find(text = 'Aired:').next_element.strip()
            if len(date)>15:
                rl=datetime.strptime(date.split(" to ")[0], '%b %d, %Y' )
            else:
                rl=datetime.strptime(date, '%b %d, %Y' )
            release.append(rl)
        except:
            release.append('NA')


        #END_DATE
        try:
            date = anime.find(text = 'Aired:').next_element.strip()
            if len(date)>15 and date.split(" to ")[1] != "?":
                e=datetime.strptime(date.split(" to ")[1], '%b %d, %Y' )
            else:
                e=pd.to_datetime(np.NaN, errors='coerce')
            end.append(e)
        except:
            end.append('NA')


        #NUM_MEMBER
        try:
            num_mem = anime.find(text = 'Members:').next_element
            nm=int(num_mem.replace('n', '').replace(',', '').strip())
            numMembers.append(nm)
        except:
            numMembers.append(0)


        #SCORE
        try:
            s=anime.find(text = 'Score:').find_next('span').contents
            s=float(s[0])
            score.append(s)
        except:
            score.append(None)


        #USERS
        try:
            us = anime.find(text = 'Score:').find_next('span').find_next('span').contents
            u=int(us[0])
            users.append(u)
        except:
            users.append(0)


        #RANK
        try:
            rk = anime.find(text = 'Ranked:').next_element
            rk=int(rk.replace('\n', '').replace('#', '').strip())
            rank.append(rk)
        except:
            rank.append(None)


        #POPULARITY
        try:
            pop = anime.find(text='Popularity:').next_element
            pop=int(pop.replace("\n","").replace('#', '').strip())
            popularity.append(pop)
        except:
            popularity.append(None)


        #DESCRIPTION
        try:
            des=anime.find(text = 'Synopsis:').find_next('p').text
            des=des.replace("\n","")
            description.append(des)
        except:
            description.append('NA')


        #RELATED
        list = anime.find(text = 'Related Anime')
        rel = []

        if(list != None):

            try:    
                list = list.find_next('list')
                list = list.find_all('a')

                for t in list:
                    rel.append(t.text)

            except:
                for t in list:
                    rel.append('NA')

        related.append(rel)

        #CHARACTER
        c = []
        try:
            list = anime.find(text = 'Characters & Voice Actors').find_next('div')
            list = list.find_all('list')

            for t in list:
                people = t.find_all('h3')
                for p in people:
                    c.append(p.text)
        except:
            c.append('NA')
            
        characters.append(c)


        #VOICES
        v=[]
        try:
            list = anime.find(text = 'Characters & Voice Actors').find_next('div')
            list = list.find_all('h3')

            for t in list:
                people = el.find_next('list')
                for person in people:
                    v.append(person.find('a').text)
        except:
            v.append('NA')

        voices.append(v)

        #STAFF
        st = []
        try:
            list = anime.find_all(text = 'Staff')[1].find_next("div", {"class": "detail-characters-list clearfix"})
            
            if(list != None):    
                table = list.find_all("table")
                for t in table:
                    i = t.find_all("td")[1]
                    p = [i.find("a").text, i.find("small").text]
                    st.append(p)
        except:
            st.append(p)

        staff.append(st)

In [ ]:
#DO NOT EXECUTE THIS CELL#

col = ['animeTitle', 'animeType', 'animeNumEpisode','releaseDate', 'endDate', 'animeNumMembers', 'animeScore', 'animeUsers', 
       'animeRank', 'animePopularity', 'animdeDescription', 'animeRelated', 'animeCharacters', 'animeVoices', 'animeStaff']

types = {'animeTitle' : 'object', 
         'animeType' : 'object', 
         'animeNumEpisode' : 'int64',
         'releaseDate' : 'datetime64', 
         'endDate' : 'datetime64', 
         'animeNumMembers' : 'int64', 
         'animeScore' : 'float64',
         'animeUsers' : 'int64', 
         'animeRank' : 'int64',
         'animePopularity' : 'int64',
         'animdeDescription' : 'object',
         'animeRelated' : 'object',
         'animeCharacters' : 'object',
         'animeVoices' : 'object',
         'animeStaff' : 'object'}

data = list(zip(title,types,numEpisode,release,end,numMembers,score,users,rank,popularity,description,related,characters,voices,staff))

dataset = pd.DataFrame(data, columns = col).astype(dtype = types)  

for i in range(len(dataset)):
    with open('C:/Users/marti/Desktop/HW3/tsv/anime_'+str(i)+'.tsv', 'w') as file:
        tsv_writer = csv.writer(file, delimiter='\t')
        tsv_writer.writerow([x for x in dataset.columns]) 
        tsv_writer.writerow(x for x in dataset.iloc[i]) 



# 2. Search engine

In [1]:
import os
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import math

# Get a list of the animes
animes = os.listdir('C:/Users/marti/Desktop/tsv_files_full')

# Save column names
names = ['Title', 'Type', 'Episodes', 'Members', 'Score', 'Users', 'Rank', 'Popularity', 'Description', 'Related', 'Characters', 'Voices', 'Staff', 'release', 'end']

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/marti/Desktop/tsv_files_full'

### 2.0.1 Remove stopwords

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
def removeStopwords(anime):
    
    # Save the English stopwords in a variable
    en_stops = set(stopwords.words('english'))
    
    for column in anime:
        processed_prop = ''
        for prop in anime[column]:
            if type(prop) is str:
                for word in prop.split():
                    if word not in en_stops:
                        processed_prop += word + ' '
                processed_df.at[0, column] = processed_prop
            else:
                processed_df.at[0, column] = prop
                
    return processed_df

### 2.0.2 Remove punctuation

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
def removePunctuation(anime):
    
    for column in anime:
        for prop in anime[column]:
            words = nltk.word_tokenize(str(prop))
            processed = [word for word in words if word.isalnum()]
            processed_df.at[0, column] = ' '.join(processed)
            
    return processed_df

### 2.0.3 Stemming

In [8]:
def stemming(anime):

    ps = PorterStemmer()

    for column in anime:
        processed_prop = ''
        for prop in anime[column]:
            if type(prop) is str:
                for word in prop.split():
                    processed_prop += ps.stem(word) + ' '
                processed_df.at[0, column] = processed_prop
            else:
                processed_df.at[0, column] = prop

    return processed_df

In [9]:
for anime_i in range(len(animes)):
    
    anime = pd.read_csv('C:/Users/marti/Desktop/tsv_files_full/anime_{i}.tsv'.format(i=anime_i), sep='\t')

    processed_df = pd.DataFrame(columns=names)    
    processed_df = removeStopwords(anime)
    processed_df = removePunctuation(processed_df)
    processed_df = stemming(processed_df)

    processed_df.to_csv('C:/Users/marti/Desktop/tsv_processed/anime_{i}.tsv'.format(i=anime_i), sep='\t')

### 2.1.1 Create the index

In [10]:
animes_lst = []
for anime_i in range(len(animes)):
    anime = pd.read_csv('C:/Users/marti/Desktop/tsv_processed/anime_{i}.tsv'.format(i=anime_i), sep='\t')
    animes_lst.append(anime)

In [11]:
#print(animes_lst)    

animes_df = pd.concat(animes_lst)
animes_df.drop('Unnamed: 0', axis=1, inplace=True)
animes_df.reset_index(inplace = True, drop=True)
print(animes_df.head())

                              Title Type  Episodes  Members Score    Users  \
0    fullmet alchemist brotherhood   tv         64  2675751   NaN  1622384   
1                               NaN  tv         51   483807   NaN   169476   
2  shingeki kyojin season 3 part 2   tv         10  1596039   NaN  1087519   
3                       stein gate   tv         24  2090910   NaN  1109700   
4           fruit basket the final   tv         13   275214   NaN   113310   

  Rank  Popularity                                        Description  \
0    1           3  after horrif alchemi experi goe wrong elric ho...   
1    2         337  gintoki shinpachi kagura return broke member y...   
2    3          33  restor diminish hope survey corp embark missio...   
3    4          11  mad scientist rintar okab rent room ricketi ol...   
4    5         651  year ago chines zodiac spirit god swore stay t...   

                                             Related  \
0  fullmet alchemist version fullmet

In [12]:
vocabulary = {}
inverted_index = {}
term_id = 1

for index, anime in animes_df.iterrows():

    description = anime['Description']  # 'Ciao mi chiamo Roberta'
    if type(description) is str:
        words = description.split()  # ['ciao', 'mi', 'chiamo', 'Roberta']

        for word in words:

            # Create vocabulary
            if word not in vocabulary:
                vocabulary[word] = term_id
                term_id += 1

In [19]:
for index, anime in animes_df.iterrows():

    description = anime['Description']  # 'Ciao mi chiamo Roberta'
    if type(description) is str:
        words = description.split()  # ['ciao', 'mi', 'chiamo', 'Roberta']

        for word in words:
            term_id = vocabulary[word]
            if term_id not in inverted_index:
                inverted_index[term_id] = []
            if index not in inverted_index[term_id]:
                inverted_index[term_id].append(index)

## 2.1.2 Execute query

In [35]:
animes_original = []
for anime_i in range(len(animes)):
    anime_original = pd.read_csv('C:/Users/marti/Desktop/tsv_files_full/anime_{i}.tsv'.format(i=anime_i), sep='\t')
    animes_original.append(anime_original)
animes_original = pd.concat(animes_original)
animes_original = animes_original.reset_index()

In [99]:
import csv
import numpy as np
animes_original.to_csv('C:/Users/marti/Desktop/file_originali_tsv/anime_original.csv')

In [89]:
with open('list_anime.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

In [100]:
def execute_query(query):
    term_id=0
    first=True
    query=query.split()
    q_match=pd.DataFrame(columns=['Title', 'Description', 'URL'])

    try:
        for term in query:
            if term in vocabulary:
                term_id=vocabulary[term]
            if first:
                lst_doc=inverted_index[term_id]
                first=False
            else:
                lst_doc=set(lst_doc).intersection(inverted_index[term_id])
                if len(lst_doc)==0:
                    print('no doc found')
                    break

        for i, doc in enumerate(lst_doc):
            q_match.at[i, ['Title', 'Description', 'URL']]=animes_original.iloc[doc]['Title'], animes_original.iloc[doc]['Description'], lines[doc]
            #q_match.at[i, 'Description']=animes_original.iloc[doc]['Description']
            #q_match.at[i, 'URL']=lines[doc]
        
        display(q_match)
    except:
        print("no doc found")


        

In [101]:
query="after"
execute_query(query)

,Title,Description,URL
0,Fullmetal Alchemist: Brotherhood,"[""After a horrific alchemy experiment goes wro...",https://myanimelist.net/anime/5114/Fullmetal_A...
1,Gintama.,"[""After joining the resistance against the bak...",https://myanimelist.net/anime/34096/Gintama\n
2,Clannad: After Story,"[<i>Clannad: After Story</i>, ', the sequel to...",https://myanimelist.net/anime/4181/Clannad__Af...
3,Gintama.: Shirogane no Tamashii-hen,"[""After the fierce battle on Rakuyou, the unto...",https://myanimelist.net/anime/36838/Gintama__S...
4,Cowboy Bebop,"['In the year 2071, humanity has colonized sev...",https://myanimelist.net/anime/1/Cowboy_Bebop\n
...,...,...,...
745,Chu^2,"['Wicked Trio 1 - The Affair ', '\n', '\r\nIt\...",https://myanimelist.net/anime/5570/Binetsu_Him...
746,The Satisfaction,['Reiko is tormented by a nightmare of a demon...,https://myanimelist.net/anime/19747/Dakaretai_...
747,Rhythm: Koi no Rhythm,"[""Hiro is a high school student with a very st...",https://myanimelist.net/anime/5571/Shining_May\n
748,Kunoichi Gakuen Ninpouchou,"[""As night arrives the shadows of Koga fall on...",https://myanimelist.net/anime/4575/Junk_Story_...


## 2.2 Conjunctive query & Ranking score

Vogliamo un inverted index dove ad ogni term id sia associata una lista di coppie composte dagli anime in cui è contenuta la parola e il suo TF-IFD score.

Il problema sta nel fatto che i dataframe di vocabulary e inverted index andrebbero ritrasformati in dizionari. C'è la funzione DataFrame.to_dict, ma non ho capito come specificare l'header

In [ ]:
vocabulary = pd.read_csv('/home/roberta/dataScience/01/Algorithmic Methods of Data Mining and Laboratory/hw3/vocabulary.csv', header=None)
inverted_index = pd.read_csv('/home/roberta/dataScience/01/Algorithmic Methods of Data Mining and Laboratory/hw3/inverted_index.csv', header=None)
"""
animes_df = pd.concat(animes_lst)
animes_df.drop('Unnamed: 0', axis=1, inplace=True)
animes_df.reset_index(inplace = True, drop=True)
"""
animes_df = pd.read_csv('/path')  # dato che l'hai salvato

second_inverted_index = {}

for index, anime in animes_df.iterrows():

    vocabulary_occurrences = {}
    description = anime['Description']  # 'Ciao mi chiamo Roberta'
    occurrence = 0

    if type(description) is str:
        words = description.split()  # ['Ciao', 'mi', 'chiamo', 'Roberta']

        # For each word in the description
        for word in words:

            # Fill the vocabulary with the occurrence
            vocabulary_occurrences[word] = occurrence + 1
            occurrence += 1

    tot_n = len(vocabulary_occurrences)
    # For each word in the vocabulary with the occurrences
    for word in vocabulary_occurrences:
        # Take the term id from the already made vocabulary
        term_id = vocabulary[word]

        # Calculate term frequency = # word appears in doc / # tot words
        word_n = vocabulary_occurrences[word]
        tf = word_n / tot_n

        # Calculate Inverse Data Frequency = log(# tot docs / # doc containing term i
        tot_docs = len(animes)
        doc_n = len(inverted_index[term_id])
        idf = math.log(tot_docs / doc_n, 10)

        # Calculate the TF-IDF score
        tfIdf = tf * idf

        # Fill the second inverted index
        if term_id not in second_inverted_index:
            second_inverted_index[term_id] = []
        if index not in second_inverted_index[term_id]:
            second_inverted_index[term_id].append((index, tfIdf))

print(second_inverted_index)

## 5. Algorithmic question

In the given problem, the goal is to choose the maximum number of appointments such as the appointments choosen are not consecutive. It means that the personal trainer want needs a break between appointments and so he can't accept two consecutive request. Obviously, all the request are consider in chronological order. The input data is just a list of the requested appointment, and we want to maximize the sum of value (not adiacent) contained in the list.

One solution could be to followed:


In [13]:
def max_sum(app):
    l1 = []
    l2 = []
    tmp1 = 0
    tmp2 = 0
    for i in range(len(app)):
        if ((i % 2) == 0):
            tmp1 += app[i]
            l1.append(app[i])
        else:
            tmp2 += app[i]
            l2.append(app[i])
    if (tmp1 > tmp2):
        return l1, tmp1
    else:
        return l2, tmp2

In [14]:
app = [30, 40, 25, 50, 30, 20] 
print('Appointments required:', app, '\n') 

lst, duration=max_sum(app)
print('List of appointment accepted: ', lst)
print('total duration (of the accepted appointment): ', duration)

Appointments required: [30, 40, 25, 50, 30, 20] 

List of appointment accepted:  [40, 50, 20]
total duration (of the accepted appointment):  110


In this program we observe each element of the list twice (one from the even index, and one from the odd index), both of this going through the list with a step of 2. Thenwe add the odd and even values ​​respectively, and then compare them. At the end, we choose the path that maximize the sum.